Notebook para filtrar a base que será usada no backtesting inicial da Magic Formula

In [7]:
import pandas as pd
pd.options.display.max_colwidth = 20
pd.options.display.max_rows = 4

In [8]:
# Lendo a base -> são mais de 10 milhões de linhas contábeis!
# cols_data = ["doc_env", "doc_ref", "per_ini", "per_fim"]
# df = pd.read_csv("DATA_FILE_PATH", parse_dates=cols_data)
df = pd.read_parquet("base.parquet")
df

,cia_id,cia_nome,doc_id,doc_arq,doc_tp,doc_ver,doc_env,doc_ref,per_ini,per_fim,per_ref,dem_tp,conta_id,conta_fixa,conta_desc,conta_valor
0,3,CIA MODELO,54947,0000032016033130...,ITR,1,2016-04-02 12:04:12,2016-03-31,2016-01-01,2016-03-31,0,IND,3.01,1,Receita de Venda...,100000000.0
1,3,CIA MODELO,54947,0000032016033130...,ITR,1,2016-04-02 12:04:12,2016-03-31,2016-01-01,2016-03-31,0,IND,3.02,1,Custo dos Bens e...,-10000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10508137,26310,CIA. SECURITIZAD...,108527,0263102021063030...,ITR,1,2021-10-08 15:53:04,2021-06-30,2021-01-01,2021-06-30,0,IND,9.01.01,1,Ações Ordinárias...,5000.0
10508138,26310,CIA. SECURITIZAD...,108527,0263102021063030...,ITR,1,2021-10-08 15:53:04,2021-06-30,2021-01-01,2021-06-30,0,IND,9.01.03,1,Total de Ações (...,5000.0


In [9]:
# Versão inicial: somente dados auditados serão usados -> (DFP) 
df.query('doc_tp == "DFP"', inplace=True)
# Remover colunas que não serão usadas no backtesting
# doc_ref -> já está implícito no período inicial e no final
# doc_ver e doc_id -> o controle da versão do documento será feito pela horário
# de envio e pelo empresa
df.drop(columns=["doc_tp", "doc_arq", "doc_ref", "doc_ver", "doc_id"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,per_ref,dem_tp,conta_id,conta_fixa,conta_desc,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2008-01-01,2008-12-31,-2,IND,1,1,Ativo Total,184750000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2008-01-01,2008-12-31,-2,CON,1,1,Ativo Total,204561000.0
...,...,...,...,...,...,...,...,...,...,...,...
7028416,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,0,IND,9.01.01,1,Ações Ordinárias...,12504967.0
7028417,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,0,IND,9.01.03,1,Total de Ações (...,12504967.0


In [10]:
# Iremos simular somente com dados consolidados (CON)
# Os dados com o número de ações foram estão no balanço individual (IND)
# A conta que interessa é a "9.01.03" -> "Total de Ações (Capital Integralizado)"
df.query('dem_tp == "CON" or conta_id == "9.01.03"', inplace=True)
# Remover coluna que não será mais usada no backtesting
df.drop(columns=["dem_tp"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,per_ref,conta_id,conta_fixa,conta_desc,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2008-01-01,2008-12-31,-2,1,1,Ativo Total,204561000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2008-01-01,2008-12-31,-2,1.01,1,Ativo Circulante,169771000.0
...,...,...,...,...,...,...,...,...,...,...
2902211,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,0,7.08.04.03,1,Lucros Retidos /...,-54161000.0
2902212,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,0,9.01.03,1,Total de Ações (...,12504967.0


* (pág. 138) *For purposes of the study, earnings-related numbers were based on the latest 12-month period, balance sheet items were based on the most recent balance sheet, and market prices were based on the most recent closing price. Utilities, financial stocks and companies where we could not be certain that the information in the database was timely or complete were eliminated. Adjustments were also made for certain non-interest bearing liabilities. The study was structured so that an average of 30 stocks was held during the study period. Stocks with only limited liquidity were eliminated from the study. Market capitalizations were determined based on 2003 dollars. Both the number of companies in each decile as well as the number of companies in each market capitalization group fluctuated as the number of companies in the database varied during the study period.

In [11]:
# Pelo que está no livro, somentes os indicadores dos últimos 12 meses serão usados. 
# Logo, podemos descartar períodos que não são o corrente -> per_ref == 0
df.query('per_ref == 0', inplace=True)
# Remover coluna que não será mais usada no backtesting
df.drop(columns=["per_ref"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,conta_id,conta_fixa,conta_desc,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1,1,Ativo Total,278858000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01,1,Ativo Circulante,220365000.0
...,...,...,...,...,...,...,...,...,...
1036943,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,7.08.04.03,1,Lucros Retidos /...,-54161000.0
1036944,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,9.01.03,1,Total de Ações (...,12504967.0


Plano de contas da CVM (parte inicial "conta_id"):
- 1 -> Balance Sheet - Assets
- 2 -> Balance Sheet - Liabilities and Shareholders’ Equity
- 3 -> Income
- 4 -> Comprehensive Income
- 5 -> Changes in Equity
- 6 -> Cash Flow (Indirect Method)
- 7 -> Added Value

Logo, para usar a fórmula precisamos filtrar as empresas por:
- Liquidez mínima (o livro filtra por valor de mercado no lugar de liquidez)
- Excluir empresas do setor financeiro e de seguros
- ROIC = EBIT / Invested Capital
- Invested Capital = Net Working Capital + Net Fixed Assets
- Earnings yield = EBIT / Enterprise Value*
    - Enterprise Value (EV) = market value of equity + net interest-bearing debt
    - Invested Capital (Damodaran) = PL + Dívida - Caixa (calculados pelo valor contábil)
- Desmembrando os indicadores:
    - EBIT = lucro antes dos juros e impostos -> 3.05
    - Market value of equity = núm. de ações x preço da ação -> 9.01.03
    - Net interest-bearing debt = dívida líquida
        - total_cash = df.loc["1.01.01"] + df.loc["1.01.02"]
        - total_debt = df.loc["2.01.04"] + df.loc["2.02.01"]
        - net_debt = total_debt - total_cash
    - PL -> df.loc["2.03"]

In [12]:
# Filtrar as contas que serão usadas
contas = ["1.01.01", "1.01.02", "2.01.04", "2.02.01", "2.03", "3.05", "9.01.03"]
df.query('conta_id == @contas', inplace=True)
df.reset_index(drop=True, inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,conta_id,conta_fixa,conta_desc,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.01,1,Caixa e Equivale...,4944000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.02,1,Aplicações Finan...,79794000.0
...,...,...,...,...,...,...,...,...,...
41628,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,3.05,1,Resultado Antes ...,-41627000.0
41629,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,9.01.03,1,Total de Ações (...,12504967.0


In [13]:
# Verificar se as contas selecionadas são fixas
df.conta_fixa.value_counts()

1    41395
0      235
Name: conta_fixa, dtype: int64

In [14]:
# Todas as empresas que contém contas não fixas nas contas selecionadas são IFs
df.query('conta_fixa == 0').cia_nome.unique()

array(['BCO BRADESCO S.A.', 'BCO BRASIL S.A.',
       'BCO ESTADO DE SERGIPE S.A. - BANESE',
       'BANESTES S.A. - BCO EST ESPIRITO SANTO',
       'BCO ESTADO DO RIO GRANDE DO SUL S.A.',
       'BCO MERCANTIL DO BRASIL S.A.', 'BCO ALFA DE INVESTIMENTO S.A.',
       'BRB BCO DE BRASILIA S.A.', 'ITAU UNIBANCO HOLDING S.A.',
       'BCO SANTANDER (BRASIL); S.A.', 'BANCO CRUZEIRO DO SUL SA',
       'BCO DAYCOVAL S.A.', 'BCO ABC BRASIL S.A.',
       'BANCO INDUSTRIAL E COMERCIAL S/A', 'BCO PAN S.A.',
       'BANCO RCI BRASIL S.A.',
       'Companhia de Crédito Financiamento e Investimento RCI Brasil',
       'BANCO BMG S/A', 'BANCO PATAGONIA SA', 'PPLA PARTICIPATIONS LTD.'],
      dtype=object)

In [15]:
# Buscar demais bancos e demais IFs
procurar = "bco |banco|crédito|mercantil|seguradora|seguro|PPLA PARTICIPATIONS"
df.query('cia_nome.str.contains(@procurar, case=False)').cia_nome.unique()

array(['BCO BRADESCO S.A.', 'BCO AMAZONIA S.A.', 'BCO BRASIL S.A.',
       'BCO ESTADO DE SERGIPE S.A. - BANESE',
       'BANESTES S.A. - BCO EST ESPIRITO SANTO',
       'BCO ESTADO DO PARA S.A.', 'BANCO BERJ S.A.',
       'BCO ESTADO DO RIO GRANDE DO SUL S.A.',
       'BCO NORDESTE DO BRASIL S.A.',
       'BCO MERCANTIL DE INVESTIMENTOS S.A.',
       'BCO MERCANTIL DO BRASIL S.A.', 'BCO ALFA DE INVESTIMENTO S.A.',
       'BMG LEASING S.A. - ARREND. MERCANTIL',
       'CIA SEGUROS ALIANCA DA BAHIA',
       'MERCANTIL BRASIL FINANC S.A. C.F.I.',
       'BFB LEASING S.A. ARRENDAMENTO MERCANTIL',
       'BRB BCO DE BRASILIA S.A.',
       'DIBENS LEASING S.A. - ARREND.MERCANTIL',
       'CCB BRASIL ARRENDAMENTO MERCANTIL S.A.',
       'MERCANTIL DO BRASIL LEASING SA', 'PORTO SEGURO S.A.',
       'PAN ARRENDAMENTO MERCANTIL S.A.', 'ITAU UNIBANCO HOLDING S.A.',
       'REC SAO BERNARDO SECURITIZADORA DE CRÉDITOS IMOBILIÁRIOS S.A.',
       'BRPR 51 SECURITIZADORA DE CRÉDITOS IMOBILIÁRIOS S.A.

In [16]:
# Remover essas empresas do dataframe
cias_remover = df.query('cia_nome.str.contains(@procurar, case=False)').cia_id.unique()
df = df[~df.cia_id.isin(cias_remover)].copy()
df.reset_index(drop=True, inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,conta_id,conta_fixa,conta_desc,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.01,1,Caixa e Equivale...,4944000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.02,1,Aplicações Finan...,79794000.0
...,...,...,...,...,...,...,...,...,...
39322,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,3.05,1,Resultado Antes ...,-41627000.0
39323,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,9.01.03,1,Total de Ações (...,12504967.0


In [17]:
# Verificar se as empresas foram removidas
df.query('cia_nome.str.contains(@procurar, case=False)').cia_nome.unique()

array([], dtype=object)

In [18]:
# Verificar novamente se as contas selecionadas são todas fixas
df.conta_fixa.value_counts()

1    39324
Name: conta_fixa, dtype: int64

In [19]:
# Remover a coluna conta fixa da base, pois não será mais usada
df.drop(columns=["conta_fixa"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,conta_id,conta_desc,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.01,Caixa e Equivale...,4944000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.02,Aplicações Finan...,79794000.0
...,...,...,...,...,...,...,...,...
39322,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,3.05,Resultado Antes ...,-41627000.0
39323,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,9.01.03,Total de Ações (...,12504967.0


In [20]:
# Remover a descrição do código contábil para a operação futura de unstack
df.drop(columns=["conta_desc"], inplace=True)
df

,cia_id,cia_nome,doc_env,per_ini,per_fim,conta_id,conta_valor
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.01,4944000.0
1,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,1.01.02,79794000.0
...,...,...,...,...,...,...,...
39322,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,3.05,-41627000.0
39323,6815,IGB ELETRONICA S...,2022-05-20 20:14:50,2021-01-01,2021-12-31,9.01.03,12504967.0


In [21]:
# Unstack -> escolher as colunas que irão para o multiindex
colunas_index = df.columns[:-1].to_list()
colunas_index

['cia_id', 'cia_nome', 'doc_env', 'per_ini', 'per_fim', 'conta_id']

In [22]:
# Passar o dataframe para multiindex
df = df.set_index(colunas_index).sort_index()
df

conta_valor
cia_id cia_nome             doc_env             per_ini    per_fim    conta_id             
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31 1.01.01     4944000.0
                                                                      1.01.02    79794000.0
...                                                                                     ...
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31 9.01.03   425555795.0
                            2021-08-11 11:24:53 2018-01-01 2018-12-31 9.01.03   425555795.0

[39324 rows x 1 columns]

In [23]:
# Fazer o unstack do dataframe com base na última coluna do índice (conta_id)
df = df.unstack(level=- 1, fill_value=0)
df

conta_valor  \
conta_id                                                                  1.01.01   
cia_id cia_nome             doc_env             per_ini    per_fim                  
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31   4944000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31   4944000.0   
...                                                                           ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31         0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31         0.0   

                                                                                   \
conta_id                                                                  1.01.02   
cia_id cia_nome             doc_env             per_ini    per_fim                  
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  79794000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  79794000.0   
...                                                                           ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31         0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31         0.0   

                                                                                   \
conta_id                                                                  2.01.04   
cia_id cia_nome             doc_env             per_ini    per_fim                  
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  61484000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  61485000.0   
...                                                                           ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31         0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31         0.0   

                                                                                  \
conta_id                                                                 2.02.01   
cia_id cia_nome             doc_env             per_ini    per_fim                 
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  4273000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  4273000.0   
...                                                                          ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31        0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31        0.0   

                                                                                    \
conta_id                                                                      2.03   
cia_id cia_nome             doc_env             per_ini    per_fim                   
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  142805000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  142082000.0   
...                                                                            ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31          0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31          0.0   

                                                                                   \
conta_id                                                                     3.05   
cia_id cia_nome             doc_env             per_ini    per_fim                  
94     PANATLANTICA S.A.    2011-03-31 10:16:48 2010-01-01 2010-12-31  24777000.0   
                            2011-04-01 17:31:56 2010-01-01 2010-12-31  21093000.0   
...                                                                           ...   
503711 Óleos de Palma S.... 2017-03-20 18:30:43 2015-01-01 2015-12-31         0.0   
                            2021-08-11 11:24:53 2018-01-01 2018-12-31 

In [24]:
# Remover o multiindex do dataframe
df.columns = df.columns.droplevel(0)
df.reset_index(inplace=True)
df

conta_id,cia_id,cia_nome,doc_env,per_ini,per_fim,1.01.01,1.01.02,2.01.04,2.02.01,2.03,3.05,9.01.03
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,4944000.0,79794000.0,61484000.0,4273000.0,142805000.0,24777000.0,8856000.0
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,4944000.0,79794000.0,61485000.0,4273000.0,142082000.0,21093000.0,8856000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9439,503711,Óleos de Palma S...,2017-03-20 18:30:43,2015-01-01,2015-12-31,0.0,0.0,0.0,0.0,0.0,0.0,425555795.0
9440,503711,Óleos de Palma S...,2021-08-11 11:24:53,2018-01-01,2018-12-31,0.0,0.0,0.0,0.0,0.0,0.0,425555795.0


In [25]:
# Calcular os indicadores
df["total_cash"] = df["1.01.01"] + df["1.01.02"]
df["total_debt"] = df["2.01.04"] + df["2.02.01"]
df.rename(
    columns={"2.03": "equity", "3.05": "ebit", "9.01.03": "shares_outstanding"},
    inplace=True
)
df["invested_capital"] = df["equity"] + df["total_debt"] - df["total_cash"]
df["roic"] = df["ebit"] / df["invested_capital"]
df.drop(columns=["1.01.01", "1.01.02", "2.01.04", "2.02.01"], inplace=True)
df

conta_id,cia_id,cia_nome,doc_env,per_ini,per_fim,equity,ebit,shares_outstanding,total_cash,total_debt,invested_capital,roic
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,142805000.0,24777000.0,8856000.0,84738000.0,65757000.0,123824000.0,0.200099
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,142082000.0,21093000.0,8856000.0,84738000.0,65758000.0,123102000.0,0.171346
...,...,...,...,...,...,...,...,...,...,...,...,...
9439,503711,Óleos de Palma S...,2017-03-20 18:30:43,2015-01-01,2015-12-31,0.0,0.0,425555795.0,0.0,0.0,0.0,NaN
9440,503711,Óleos de Palma S...,2021-08-11 11:24:53,2018-01-01,2018-12-31,0.0,0.0,425555795.0,0.0,0.0,0.0,NaN


In [26]:
df.columns

Index(['cia_id', 'cia_nome', 'doc_env', 'per_ini', 'per_fim', 'equity', 'ebit',
       'shares_outstanding', 'total_cash', 'total_debt', 'invested_capital',
       'roic'],
      dtype='object', name='conta_id')

In [27]:
# Remover os indicadores intermediários que não serão usados no backtesting
df.drop(
    columns=['equity', 'total_cash', 'total_debt', 'invested_capital'],
    inplace=True
)
# Reorganizar a ordem das colunas
colunas = df.columns[:-3].to_list() + ['shares_outstanding', 'ebit', 'roic']
df = df[colunas].copy()
df

conta_id,cia_id,cia_nome,doc_env,per_ini,per_fim,shares_outstanding,ebit,roic
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,8856000.0,24777000.0,0.200099
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,8856000.0,21093000.0,0.171346
...,...,...,...,...,...,...,...,...
9439,503711,Óleos de Palma S...,2017-03-20 18:30:43,2015-01-01,2015-12-31,425555795.0,0.0,NaN
9440,503711,Óleos de Palma S...,2021-08-11 11:24:53,2018-01-01,2018-12-31,425555795.0,0.0,NaN


In [28]:
# Remover ebit negativos ou próximos de 0 (logo, ROIC negativo ou próximo de 0) da base,
# pois essas empresas não serão selecionadas pela fórmula
df.query('ebit >= 0.001', inplace=True)
df

conta_id,cia_id,cia_nome,doc_env,per_ini,per_fim,shares_outstanding,ebit,roic
0,94,PANATLANTICA S.A.,2011-03-31 10:16:48,2010-01-01,2010-12-31,8856000.0,24777000.0,0.200099
1,94,PANATLANTICA S.A.,2011-04-01 17:31:56,2010-01-01,2010-12-31,8856000.0,21093000.0,0.171346
...,...,...,...,...,...,...,...,...
9433,80195,"G2D Investments,...",2021-03-12 18:33:08,2020-01-01,2020-12-31,66025325.0,92232000.0,0.191407
9437,90212,Multilaser Indus...,2018-05-30 15:43:03,2017-01-01,2017-12-31,216075329.0,241375000.0,0.305856


In [29]:
# Verificar os indicadores da Petro
# O total de ações da publicação enviada em 2011-02-25 21:33:09 foi corrigido
# na versão seguinte
df.query('cia_id == 9512')

conta_id,cia_id,cia_nome,doc_env,per_ini,per_fim,shares_outstanding,ebit,roic
1500,9512,PETROLEO BRASILE...,2011-02-25 21:33:09,2010-01-01,2010-12-31,1.304450e+13,4.557459e+10,0.122579
1501,9512,PETROLEO BRASILE...,2011-03-04 16:04:52,2010-01-01,2010-12-31,1.304450e+10,4.557459e+10,0.122579
...,...,...,...,...,...,...,...,...
1517,9512,PETROLEO BRASILE...,2021-03-22 22:06:01,2020-01-01,2020-12-31,1.304450e+10,4.962100e+10,0.077603
1518,9512,PETROLEO BRASILE...,2022-02-23 22:36:50,2021-01-01,2021-12-31,1.304450e+10,2.108310e+11,0.321703


In [30]:
df.to_csv("magic_data.csv")